# 🧠 AI Chatbot - Deep Learning Training Notebook

Notebook này giúp bạn huấn luyện mô hình LSTM/GRU cho Chatbot trên Google Colab.

### 📝 Hướng dẫn sử dụng:
1.  **Upload file `train_data_full.csv`** (được tạo từ script `app/export_data.py`) lên Colab.
2.  Chạy lần lượt các ô code bên dưới (nhấn nút Play ▶️).
3.  Sau khi chạy xong, notebook sẽ tự động nén và tải về file `models.zip`.
4.  Giải nén file này và chép vào thư mục `models/` trong dự án của bạn.

In [ ]:
# 📦 1. Cài đặt thư viện cần thiết
!pip install pyvi tensorflow pandas scikit-learn

In [ ]:
# 📚 2. Import thư viện
import pandas as pd
import numpy as np
import pickle
import os
import re
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, SpatialDropout1D, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from pyvi import ViTokenizer 
from sklearn.feature_extraction.text import TfidfVectorizer

print("✅ Libraries imported successfully.")

In [ ]:
# 🧹 3. Định nghĩa Class Preprocessor (Copy từ app/preprocess.py)
# ---------------------------------------------------------

class TextPreprocessor:
    _instance = None

    def __new__(cls):
        if cls._instance is None:
            cls._instance = super(TextPreprocessor, cls).__new__(cls)
            cls._instance._initialize()
        return cls._instance

    def _initialize(self):
        self.re_special_chars = re.compile(r'[^\w\s]')
        self.re_numbers = re.compile(r'\d+')
        self._load_dictionaries()

    def _load_dictionaries(self):
        self.VIETNAMESE_STOPWORDS = {
            'thì', 'là', 'mà', 'và', 'của', 'những', 'các', 'như', 'thế', 'nào', 
            'được', 'về', 'với', 'trong', 'có', 'không', 'cho', 'tôi', 'bạn', 
            'cậu', 'tớ', 'mình', 'nó', 'hắn', 'gì', 'cái', 'con', 'người', 
            'sự', 'việc', 'đó', 'đây', 'kia', 'này', 'nhé', 'ạ', 'ơi', 'đi', 
            'làm', 'khi', 'lúc', 'nơi', 'tại', 'đã', 'đang', 'sẽ', 'muốn', 
            'phải', 'biết', 'hãy', 'rồi', 'chứ', 'nhỉ'
        }

    def preprocess_text(self, text: str) -> str:
        if not text: return ""
        text = text.lower()
        text = self.re_special_chars.sub('', text)
        text = self.re_numbers.sub('', text)
        tokenized_text = ViTokenizer.tokenize(text)
        tokens = tokenized_text.split()
        filtered_tokens = [
            word for word in tokens 
            if word not in self.VIETNAMESE_STOPWORDS and len(word) > 1
        ]
        return ' '.join(filtered_tokens)

# Khởi tạo Preprocessor
preprocessor = TextPreprocessor()
def preprocess_text(text: str) -> str:
    return preprocessor.preprocess_text(text)

print("✅ Preprocessor initialized.")

In [ ]:
# 🏗️ 4. Định nghĩa Model Deep Learning (Copy từ app/dl_model.py)
# ---------------------------------------------------------

def create_model(vocab_size, embedding_dim, max_length, num_classes):
    model = Sequential()
    model.add(Embedding(vocab_size, embedding_dim, input_length=max_length))
    model.add(SpatialDropout1D(0.2))
    model.add(Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

print("✅ Model architecture defined.")

In [ ]:
# 🔥 5. Huấn luyện Mô hình (Logic từ app/train_dl.py)
# ---------------------------------------------------------

# Cấu hình
MAX_NUM_WORDS = 5000
MAX_SEQUENCE_LENGTH = 100
EMBEDDING_DIM = 100
EPOCHS = 10
BATCH_SIZE = 32
MODEL_PATH = 'models/dl_model.h5'
TOKENIZER_PATH = 'models/tokenizer.pickle'
LABEL_ENCODER_PATH = 'models/label_encoder.pickle'

def train_dl_model():
    print("🚀 Starting Deep Learning Model Training...")

    # 1. Load Data
    if not os.path.exists('train_data_full.csv'):
        print("❌ Error: train_data_full.csv not found. Please upload it!")
        return

    df = pd.read_csv('train_data_full.csv')
    
    # 2. Preprocess Data
    print("🧹 Preprocessing text...")
    df['clean_text'] = df['question'].apply(preprocess_text)
    
    # 3. Tokenization & Padding
    print("🔠 Tokenizing...")
    tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\\]^`{|}~\t\n', lower=True)
    tokenizer.fit_on_texts(df['clean_text'].values)
    
    X = tokenizer.texts_to_sequences(df['clean_text'].values)
    X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)

    # 4. Encode Labels
    print("🏷️ Encoding labels...")
    if 'topic' not in df.columns:
        print("❌ Error: 'topic' column missing in dataset!")
        return
    else:
        target_column = 'topic'
        
    le = LabelEncoder()
    Y = le.fit_transform(df[target_column])
    num_classes = len(le.classes_)
    print(f"Number of classes: {num_classes}")

    # 5. Build Model
    print("🏗️ Building model...")
    model = create_model(MAX_NUM_WORDS, EMBEDDING_DIM, MAX_SEQUENCE_LENGTH, num_classes)
    model.summary()

    # 6. Train Model
    print("🔥 Training...")
    model.fit(X, Y, epochs=EPOCHS, batch_size=BATCH_SIZE, validation_split=0.1, verbose=1)

    # 7. Save Artifacts
    print("💾 Saving artifacts...")
    if not os.path.exists('models'):
        os.makedirs('models')
        
    model.save(MODEL_PATH)
    
    with open(TOKENIZER_PATH, 'wb') as handle:
        pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    with open(LABEL_ENCODER_PATH, 'wb') as handle:
        pickle.dump(le, handle, protocol=pickle.HIGHEST_PROTOCOL)

    print("✅ Training complete! Model saved to models/")

# Run Training
train_dl_model()

In [ ]:
# 📦 6. Nén và Tải về Model
!zip -r models.zip models/
from google.colab import files
files.download('models.zip')